In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rc('axes', unicode_minus=False)
%matplotlib inline

np.random.seed(123)

In [3]:
train = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/job_train.csv')
test = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/job_test.csv')
df = pd.concat([train,test])
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14668 entries, 0 to 14667
Columns: 109 entries, STU_ID to X1STU30OCC2
dtypes: float64(36), int64(73)
memory usage: 12.2 MB


In [4]:
df.X1STU30OCC2.nunique()

21

In [5]:
df.X1STU30OCC2.value_counts()

29.0    3479
27.0    2088
19.0     844
17.0     828
23.0     657
25.0     616
33.0     478
55.0     461
11.0     369
39.0     286
15.0     280
49.0     254
35.0     232
13.0     217
21.0     165
47.0     134
53.0     114
41.0      88
51.0      69
31.0      43
45.0      38
Name: X1STU30OCC2, dtype: int64

In [6]:
df = df.where(~df.isin([-9,-8,-7]), np.nan)


In [7]:
null_check = df.isnull().sum()

In [8]:
sum_df = df.describe().T.iloc[:,3:]
sum_df['null_check'] = null_check
sum_df.to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/sum_df.csv')

In [9]:
sum_df 

,min,25%,50%,75%,max,null_check
STU_ID,10001.00,16317.75,22661.50,28909.75,35205.00,0
X1SEX,1.00,1.00,2.00,2.00,2.00,0
X1RACE,1.00,5.00,8.00,8.00,8.00,0
X1HISPANIC,0.00,0.00,0.00,0.00,1.00,0
X1WHITE,0.00,0.00,1.00,1.00,1.00,118
...,...,...,...,...,...,...
X1COUPERCOU,-6.43,-0.69,0.26,1.02,1.15,1574
X1COUPERPRI,-5.51,-0.68,0.20,1.10,1.10,1912
X1AQSTAT,1.00,1.00,1.00,1.00,8.00,0
X1AQDATE,200910.00,200911.00,201001.00,201001.00,201001.00,746


In [10]:
##nullcheck 어떻게 처리할지 정해야함

nullp = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/null_pre.csv')
nullp = nullp.dropna().reset_index(drop=True)
nullp = nullp.set_index('variables')['null_pre']
nullp

variables
STU_ID         0.00
X1SEX          0.00
X1RACE         0.00
X1HISPANIC     0.00
X1WHITE        0.00
               ... 
X1COUPERTEA    0.03
X1COUPERCOU    0.26
X1COUPERPRI    0.20
X1AQSTAT       0.00
X1STU30OCC2    0.00
Name: null_pre, Length: 103, dtype: float64

In [11]:
#csv에 작성한 null값을 읽어서 채움
df = df[nullp.index]
df = df.fillna(nullp)

In [12]:
df.isnull().sum().sum()

0

In [13]:
df['birthyear'] = (df['X1STDOB']/100).astype('int')
df['birthyear'] = (df.birthyear % 1990).astype('int')
df['birthmonth'] = (df['X1STDOB']%100).astype('int')

<ipython-input-13-c05ad4bb4e5f>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['birthyear'] = (df['X1STDOB']/100).astype('int')
<ipython-input-13-c05ad4bb4e5f>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['birthmonth'] = (df['X1STDOB']%100).astype('int')


In [14]:
test = df[df.X1STU30OCC2==0].reset_index(drop=True)
test = test.drop(columns = ['X1STU30OCC2'])
train = df[df.X1STU30OCC2!=0].reset_index(drop=True)

In [15]:
idx2job = train['X1STU30OCC2'].value_counts().reset_index().astype('int')
idx2job = idx2job[['index']]
idx2job.columns = ['job']
idx2job.to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/idx2job.csv')
idx2job = idx2job['job'].to_dict()
job2idx = {v:i for i,v in idx2job.items()}

In [16]:
train['X1STU30OCC2'] = train['X1STU30OCC2'].replace(job2idx).astype('int')

In [17]:
train.to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/train_ver02.csv', index=False)
test.to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/test_ver02.csv', index=False)

In [18]:
train.shape

(11740, 105)

In [19]:
test.shape

(2928, 104)

In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [21]:
train = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/train_ver02.csv', index_col='STU_ID')
test = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/test_ver02.csv', index_col='STU_ID')
idx2job = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/idx2job.csv', index_col=0)
idx2job = idx2job['job'].to_dict()
train.head()

,X1SEX,X1RACE,X1HISPANIC,X1WHITE,X1BLACK,X1DUALLANG,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,...,X1LOCALE,X1REGION,X1SCHOOLCLI,X1COUPERTEA,X1COUPERCOU,X1COUPERPRI,X1AQSTAT,X1STU30OCC2,birthyear,birthmonth
STU_ID,,,,,,,,,,,,,,,,,,,,,
10002,2,8,0,1.0,0.0,1.0,199511.0,-0.2956,0.2696,35.8045,...,4,1,0.32,-0.80,1.15,-1.61,1,0,5,11
10003,2,3,0,0.0,1.0,1.0,199506.0,1.2997,0.2555,56.0477,...,2,4,-0.61,0.81,-0.15,0.00,1,5,5,6
10004,2,8,0,1.0,0.0,1.0,199505.0,-0.1427,0.2649,38.0888,...,2,3,-2.14,0.03,0.26,0.20,1,0,5,5
10005,1,8,0,1.0,0.0,1.0,199505.0,1.1405,0.2312,54.1762,...,1,3,-0.65,-0.25,0.14,0.05,1,10,5,5
10006,2,8,0,1.0,0.0,1.0,199504.0,0.7106,0.2567,49.0531,...,4,3,-0.31,0.51,-0.90,0.69,1,0,5,4


In [22]:
y = train.X1STU30OCC2
x = train.drop(columns='X1STU30OCC2')

In [23]:
forest = RandomForestClassifier(n_estimators=300, n_jobs=-1)
forest.fit(x,y)

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [24]:
# 0.37403
pred_base = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : forest.predict(test)})
pred_base['X1STU30OCC2'] = pred_base['X1STU30OCC2'].replace(idx2job)
pred_base.to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/jobrec_baseline.csv', index=False, encoding='utf-8-sig')

In [25]:
imp = forest.feature_importances_
idx = np.argsort(imp)[::-1]
labels = x.columns
pd.DataFrame({'columns':x.columns, 'imp':forest.feature_importances_}).sort_values(by='imp', ascending=False)

,columns,imp
66,X1STUEDEXPCT,0.023792
0,X1SEX,0.022960
8,X1TXMSEM,0.022859
65,X1SCHOOLENG,0.021048
54,X1SES_U,0.020892
...,...,...
71,X1PQSTAT,0.001043
69,X1TESTSTAT,0.000953
17,X1MACC,0.000866
70,X1SQSTAT,0.000682


In [26]:
x_filter = x[labels[imp>0.005]] #threshold를 줘서 피쳐 줄였음
x_filter.shape
test_filter = test[x_filter.columns]

In [27]:
forest_small = RandomForestClassifier(n_estimators=300, n_jobs=-1)
forest_small.fit(x_filter,y)

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [28]:
#0.37959
g_small = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : forest_small.predict(test_filter)})
g_small['X1STU30OCC2'] = g_small['X1STU30OCC2'].replace(idx2job)
g_small.to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/jobrec_small.csv', index=False, encoding='utf-8-sig')

In [29]:
train_new = x_filter
train_new['X1STU30OCC2'] = y.values
train_new.reset_index().to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/train_ver03.csv',index=False, encoding='utf-8-sig')
test_filter.reset_index().to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/test_ver03.csv',index=False, encoding='utf-8-sig')

<ipython-input-29-80fa6a7fdfee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_new['X1STU30OCC2'] = y.values


In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(123)

In [31]:
df = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/train_ver03.csv', index_col='STU_ID')
df.head()

,X1SEX,X1RACE,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,X1TXMTSCOR,X1TXMPROF1,X1TXMPROF2,X1TXMPROF3,...,X1LOCALE,X1REGION,X1SCHOOLCLI,X1COUPERTEA,X1COUPERCOU,X1COUPERPRI,X1AQSTAT,birthyear,birthmonth,X1STU30OCC2
STU_ID,,,,,,,,,,,,,,,,,,,,,
10002,2,8,199511.0,-0.2956,0.2696,35.8045,47.6821,0.9714,0.5204,0.1768,...,4,1,0.32,-0.80,1.15,-1.61,1,5,11,0
10003,2,3,199506.0,1.2997,0.2555,56.0477,64.2431,1.0000,0.9914,0.9838,...,2,4,-0.61,0.81,-0.15,0.00,1,5,6,5
10004,2,8,199505.0,-0.1427,0.2649,38.0888,49.2690,0.9843,0.6416,0.2459,...,2,3,-2.14,0.03,0.26,0.20,1,5,5,0
10005,1,8,199505.0,1.1405,0.2312,54.1762,62.5897,0.9999,0.9863,0.9686,...,1,3,-0.65,-0.25,0.14,0.05,1,5,5,10
10006,2,8,199504.0,0.7106,0.2567,49.0531,58.1268,0.9995,0.9539,0.8469,...,4,3,-0.31,0.51,-0.90,0.69,1,5,4,0


In [32]:
# y = df.X1STU30OCC2
# x = df.drop(columns='X1STU30OCC2')
# x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)

In [33]:
# y_train.value_counts(normalize=True)

In [34]:
# y_test.value_counts(normalize=True)

In [35]:
# import joblib
# with open('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/my_val.pkl', 'wb') as f:
#     joblib.dump({'x':x_train,
#                  'y':y_train,
#                  'val_x':x_test,
#                  'val_y':y_test},f)

In [36]:
# import pandas as pd
# import numpy as np
# import joblib

In [37]:
# with open('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/my_val.pkl', 'rb') as f:
#     data = joblib.load(f)
#     print(data.keys())
# locals().update(data)
# del data
# np.random.seed(123)

In [38]:
test = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/test_ver03.csv', index_col='STU_ID')
idx2job = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/idx2job.csv', index_col=0)
idx2job = idx2job['job'].to_dict()
test.head()

,X1SEX,X1RACE,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,X1TXMTSCOR,X1TXMPROF1,X1TXMPROF2,X1TXMPROF3,...,X1TSRESP,X1LOCALE,X1REGION,X1SCHOOLCLI,X1COUPERTEA,X1COUPERCOU,X1COUPERPRI,X1AQSTAT,birthyear,birthmonth
STU_ID,,,,,,,,,,,,,,,,,,,,,
12424,1,3,199509.0,-0.1663,0.2511,37.7431,49.0240,0.9827,0.6242,0.2338,...,-0.13,2,1,-1.19,0.16,1.15,1.10,1,5,9
14300,1,8,199503.0,-0.6779,0.2836,29.9986,43.7132,0.8859,0.2278,0.0824,...,-0.33,1,1,1.00,1.07,-0.02,1.10,1,5,3
29579,1,2,199508.0,1.1883,0.2463,54.7405,63.0867,0.9999,0.9881,0.9742,...,-0.13,2,2,0.14,0.03,0.26,0.20,1,5,8
28217,1,8,199502.0,-2.1054,0.3637,17.2372,28.8934,0.0374,0.0174,0.0284,...,-0.33,1,1,1.00,1.07,-0.02,1.10,1,5,2
11561,2,8,199504.0,0.3538,0.2378,44.6988,54.4230,0.9979,0.8838,0.6065,...,0.22,3,3,-1.54,-0.60,-1.38,-1.11,1,5,4


- baseline modeling

In [39]:
!pip3 install pycaret
!pip install xgboost
!pip install lightgbm
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.6/483.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.0 MB/s eta 0:00:0

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11740 entries, 10002 to 35205
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   X1SEX         11740 non-null  int64  
 1   X1RACE        11740 non-null  int64  
 2   X1STDOB       11740 non-null  float64
 3   X1TXMTH       11740 non-null  float64
 4   X1TXMSEM      11740 non-null  float64
 5   X1TXMSCR      11740 non-null  float64
 6   X1TXMTSCOR    11740 non-null  float64
 7   X1TXMPROF1    11740 non-null  float64
 8   X1TXMPROF2    11740 non-null  float64
 9   X1TXMPROF3    11740 non-null  float64
 10  X1TXMPROF4    11740 non-null  float64
 11  X1TXMPROF5    11740 non-null  float64
 12  X1PAR1EDU     11740 non-null  float64
 13  X1PAR1EMP     11740 non-null  float64
 14  X1PAR1OCC2    11740 non-null  float64
 15  X1PAR1RACE    11740 non-null  float64
 16  X1P2RELATION  11740 non-null  float64
 17  X1PAR2EDU     11740 non-null  float64
 18  X1PAR2OCC2    11740 no

In [41]:
from pycaret import classification as cls

# 옵션 찾아보고 인자 추가하기
cls.setup(data=df, target='X1STU30OCC2', train_size=0.8, session_id = 1234, 
          use_gpu=True, 
          verbose=True,     
          normalize = True,           # 데이터 정규화 여부
          transformation = True,      # 데이터 로그 변환 여부
          remove_multicollinearity = True, # 다중공선성 제거 여부
          multicollinearity_threshold = 0.95, # 다중공선성 제거를 위한 VIF 임계값
          feature_selection = True,  # 자동 특성 선택 사용 여부
          n_features_to_select = 0.8,  # 특성 선택을 위한 상관계수 임계값
          #fix_imbalance = True, #불균형 처리
          remove_outliers=True,
          #pca=True,
          polynomial_features = True, 
          polynomial_degree = 2
)

,Description,Value
0,Session id,1234
1,Target,X1STU30OCC2
2,Target type,Multiclass
3,Original data shape,"(11740, 69)"
4,Transformed data shape,"(11270, 55)"
5,Transformed train set shape,"(8922, 55)"
6,Transformed test set shape,"(2348, 55)"
7,Numeric features,68
8,Preprocess,True
9,Imputation type,simple


In [42]:
top_model = cls.compare_models(fold=10, n_select=2, sort='acc', include=['xgboost', 'lightgbm', 'catboost', 'et', 'rf'])

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:40:49
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Processing:   0%|          | 0/26 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
my_models = [cls.tune_model(m) for m in top_model]

In [ ]:
top_model = cls.stack_models(my_models, optimize='acc')

In [ ]:
print(top_model)

In [ ]:
final_model = cls.finalize_model(top_model)

In [ ]:
cls.save_model(final_model, 'automl_ver01')

In [ ]:
test.head()

In [ ]:
pred = cls.predict_model(final_model, data=test)
pred.head()

In [ ]:
#0.38129
pipe_base = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : pred['prediction_label']})
#pipe_base = pd.DataFrame({'STU_ID': test.index, 'X1STU30OCC2' : pipe.predict(test)})
pipe_base['X1STU30OCC2'] = pipe_base['X1STU30OCC2'].replace(idx2job)
pipe_base.to_csv('/content/drive/Othercomputers/내 노트북/study/08_DS/data/multi_classification/pipe_base.csv', index=False, encoding='utf-8-sig')